`카드명`, `카드 고유 번호`, `카드 이미지`, `혜택 요약` 크롤링

In [1]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

url = "https://www.hanacard.co.kr/OPI31000000D.web?schID=pcd&mID=OPI31000005P&CT_ID=241704030444153#none"

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

driver.implicitly_wait(20)
driver.get(url)
time.sleep(3)

card_names = []
card_urls = []
card_imgs = []
card_summary = []

tabs = driver.find_elements(By.CSS_SELECTOR, "#stc_list > li")
 
for tab in tabs:
    tab.click()
    
    time.sleep(3)
    tab_content = driver.page_source
    soup = BeautifulSoup(tab_content, 'html.parser')
    
    # 각 탭에서 카드 정보 추출 ==========
    main_area = soup.find('article', {'class': 'card_main_area'})
    card_ul = main_area.find('ul',{'class': 'card_slide_area'})
    card_li = card_ul.find_all('li', {'class': 'li'})
    
    for i in range(len(card_li)):
        # 카드 이름 
        name = card_li[i].find('dl',{'class': 'txt'}).find('dt').text
        card_names.append(name)
                
        # 카드 고유 번호
        url_btn = card_li[i].find('ul', {'class': 'btn'}).find_all('li')[1]
        a_tag = url_btn.find('a', {'class': 'btn_ty04'})

        onclick_value = a_tag.get('onclick')
        url = onclick_value.split("'")[1]
        card_urls.append(url)
        
        # 카드 이미지
        img = card_li[i].find('img')['src']
        card_imgs.append('https://www.hanacard.co.kr' + img)
        
        # 혜택 요약
        summary_div = card_li[i].find('div', {'class': 'over'})
        summary_li = summary_div.find('ul', {'class': 's1'}).find_all('li')
        summary = ''
        for i in range(len(summary_li)):
            summary += f' {i+1}. {summary_li[i].text}' 
        
        card_summary.append(summary)
        
# 웹 드라이버 종료
driver.quit()

data = {"card_name" : card_names, "card_url" : card_urls, "card_img": card_imgs, "card_summary": card_summary}
df = pd.DataFrame(data)

df.to_csv("hana_cardInfos.csv", encoding = "utf-8-sig")

In [2]:
cardInfo = pd.read_csv('./hana_cardInfos.csv')
cardInfo

,Unnamed: 0,card_name,card_url,card_img,card_summary
0,0,#MY WAY(샵 마이웨이) 카드,15444,https://www.hanacard.co.kr/ATTACH/NEW_HOMEPAGE...,"1. 대중교통,커피,편의점 건당 300원 청구할인 2. 디지털 구독, 주유/충전 ..."
1,1,#any 하나카드,15249,https://www.hanacard.co.kr/ATTACH/NEW_HOMEPAGE...,1. 디지털 구독 50% 청구할인 2. 디지털 결제 1% 청구할인 3. 국내외 전...
2,2,에너지 더블 카드,16772,https://www.hanacard.co.kr/ATTACH/NEW_HOMEPAGE...,1. 에너지 공과금 10% 청구할인 2. 일상생활 5% 청구할인 3. EV(전기차...
3,3,MULTI Living 모바일카드,12900,https://www.hanacard.co.kr/ATTACH/NEW_HOMEPAGE...,1. 온라인 쇼핑/마트 주중 최대 10% 할인 2. 주유/백화점 주말 최대 10%...
4,4,#tag1카드 Orange,8551,https://www.hanacard.co.kr/ATTACH/NEW_HOMEPAGE...,1. 커피 25~50% 할인 2. 편의점/통신요금/쇼핑 5~10% 할인 3. 대중...
5,5,#tag1카드 Navy,8553,https://www.hanacard.co.kr/ATTACH/NEW_HOMEPAGE...,1. 학원/아파트관리비 5~10% 할인 2. 통신요금/쇼핑 5~10% 할인 3. ...
6,6,국민 행복 카드,12792,https://www.hanacard.co.kr/ATTACH/NEW_HOMEPAGE...,1. 보육료/유아학비 본인부담금 할인 1만원 청구할인 2. 커피 10% 청구할인 ...
7,7,그린 카드,1215,https://www.hanacard.co.kr/ATTACH/NEW_HOMEPAGE...,"1. 에코머니 포인트 적립 2. 문화, 레저시설 최대 50% 할인 3. 대형 할인..."
8,8,하나 원큐 카드,12942,https://www.hanacard.co.kr/ATTACH/NEW_HOMEPAGE...,1. 하나원큐페이 결제 1.8% 적립 2. 국내 전가맹점 0.8% 적립 3. 해외...
9,9,대구로 카드,16797,https://www.hanacard.co.kr/ATTACH/NEW_HOMEPAGE...,1. 대구로 택시 앱 10% 청구할인 2. 대구로 앱 10% 청구할인 3. 생활밀...
